In [15]:
# 020_multi_chain_travel.py (Notebook용)

from langchain.prompts import ChatPromptTemplate
from langchain.schema import SystemMessage, HumanMessage, StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
# === 1단계 체인: 여행지 → 대표 명소 추천 ===
step1_prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content="당신은 여행 전문가입니다. 여행지에 대해 추천 명소를 제안합니다."),
    HumanMessage(content="사용자가 입력한 여행지: {destination}\n대표 명소 1가지를 추천해주세요.")
])

llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    #model="openai/gpt-oss-120b",  # Spring AI와 동일한 모델
    model="moonshotai/kimi-k2-instruct-0905",
    temperature=0.7
)

parser = StrOutputParser()  # 문자열로 출력

# --- 1단계 체인: 여행지 → 대표 명소 추천 ---
# Prompt for recommending a single, representative attraction
step1_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 여행 전문가입니다. 여행지에 대해 추천 명소를 제안합니다."),
    ("user", "사용자가 입력한 여행지: {destination}\n대표 명소 1가지를 추천해주세요.")
])
step1_chain = step1_prompt | llm | parser

# --- 2단계 체인: 명소 → 상세 정보 제공 ---
# Prompt for providing detailed information about the attraction
step2_prompt = ChatPromptTemplate.from_messages([
    ("system", """당신은 관광 안내 전문가입니다. 명소에 대한 상세 정보를 제공합니다.
    
    Format:
    명소: {attraction}
    역사: [짧고 간결한 역사 설명]
    특징: [주요 특징, 무엇을 볼 수 있는지 설명]
    방문 팁: [방문 시 유용한 팁, 예: 예매, 시간, 복장 등]
    """),
    ("user", "{attraction}")
])
step2_chain = step2_prompt | llm | parser

# --- LCEL 체인 연결 ---
# Use RunnablePassthrough and RunnableLambda to correctly link the two chains.
# This structure ensures the output of the first chain is passed as a key to the second.
full_chain = (
    {"attraction": step1_chain, "destination": RunnablePassthrough()}
    | step2_chain
)

# --- 실행 ---
destination_input = input("여행지를 입력하세요 (예: 로마, 뉴욕, 도쿄): ")

# 1단계 결과: LCEL의 invoke 메서드를 직접 호출하여 첫 번째 체인만 실행
print(f"\n1단계 추천 명소: {step1_chain.invoke({'destination': destination_input})}")

# 2단계 결과: 전체 체인을 실행하여 최종 결과 확인
print("\n2단계 명소 상세 정보:\n")
final_result = full_chain.invoke({"destination": destination_input})
print(final_result)


1단계 추천 명소: 콜로세움(Colosseum) – 로마를 대표하는 2천 년 된 원형경기장으로, 고대 로마의 역사와 웅장함을 가장 생생히 느낄 수 있는 필수 명소입니다.

2단계 명소 상세 정보:

명소: 로마에 가면 꼭 가야 할 대표 명소는 **콜로세움(Colosseo)**입니다. 2천 년 전 로마 제국의 상징이자 세계에서 가장 유명한 원형경기장으로, 내부와 지하 시설을 둘러보며 당시의 역사와 엔터테인먼트 문화를 생생히 느낄 수 있습니다.

역사: 서기 72년 황제 베스파시아누스가 착공해 80년 티투스 황제 시절 완공한 후 100년간 5만~8만 명을 수용하며 검투사·야수·해상 전투 쇼를 벌였습니다. 5세기 이후 지진·석재 약탈로 붕괴되었으나 르네상스 이후 보호·발굴되어 2007년 세계 7대 불가사의로 선정되었습니다.

특징:  
- 4층 규모 원형 아레나(경기장 바닥)와 좌석굴(관중석)이 그대로 남아 있어 원형·지하수로·승강기·야수 우리를 확인할 수 있습니다.  
- 콜로세움·포럼·팔라티노 언덕 연합 입장권으로 구 로마의 정치·종교·시민 일상을 한눈에 조망할 수 있습니다.  
- 야간 조명이 들어오면 반사등이 무대처럼 아레나를 밝혀 낮과는 다른 분위기를 제공합니다.

방문 팁:  
1. 온라인(공식 사이트·코오파티치움)에서 시간대별 입장권을 예매하면 줄을 2시간 이상 줄일 수 있습니다.  
2. 무더운 여름 11:00–15:00 피하고 이른 아침(개장 8:30) 또는 저녁(4월–9월 19:00까지)에 방문하면 사진도 잘 나옵니다.  
3. 투어는 ‘지하·3층 전망대’ 또는 ‘야간 투어’가 가장 인기 있으며, 이어폰 가이드보다 가이드 투어가 설명이 생생합니다.  
4. 가방은 반입이 제한되므로 소지품을 최소화하고, 통풍이 잘 안 되는 돌계단이 많으므로 편한 신발·모자·물병을 챙기세요.
